In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-08 09:51:21.888 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=128, worker=0)

In [4]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from loguru import logger
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import torch

from eeg_snn_encoder.encoders import PoissonEncoderExpand
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig

torch.set_float32_matmul_precision("medium")


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {"interval_freq": trial.suggest_int("interval_freq", 1, 8), "random_seed": 47}

    spike_encoder = PoissonEncoderExpand(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=20,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val_mse"),
            EarlyStopping(monitor="val_loss", mode="min", patience=5),
        ],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)
    trainer.test(lit_model, datamodule=datamodule)

    test_loss = trainer.callback_metrics["test_loss"].item()
    test_acc = trainer.callback_metrics["test_acc"]
    test_f1 = trainer.callback_metrics["test_f1"]
    test_mse = trainer.callback_metrics["test_mse"]
    test_total_spikes = trainer.callback_metrics["test_total_spikes"]

    logger.info(
        f"Encoder: Poisson Encoding,trial: {trial.number}, test_loss:{test_loss}, test_mse:{test_mse}, test_acc:{test_acc}, test_f1:{test_f1}, test_total_spikes:{test_total_spikes}"
    )

    return test_mse

In [5]:
import os

# Initialize the Optuna study
sampler = optuna.samplers.TPESampler(multivariate=True)
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(
    direction="minimize",
    study_name="model-tuning-poisson-mse",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    sampler=sampler,
    pruner=pruner,
)

/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-08 09:51:25,982] Using an existing study with name 'model-tuning-poisson-mse' instead of creating a new one.


[I 2025-05-07 16:16:59,453] Using an existing study with name 'model-tuning-poisson' instead of creating a new one.


In [6]:
study.optimize(objective, n_trials=30)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-05-08 10:58:39,129] Trial 51 failed with parameters: {'threshold': 0.011611622706112528, 'slope': 2.4966549089214625, 'beta': 0.81993281233496, 'dropout_rate1': 0.1733864931936592, 'dropout_rate2': 0.7598351914129508, 'lr': 1.0397595916540152e-05, 'weight_decay': 6.7288759707487635e-06, 'scheduler_factor': 0.690324809841639, 'scheduler_patience': 9, 'interval_freq': 6} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/jupyter-group55

NameError: name 'exit' is not defined

In [ ]:
logger.info(f"Encoder: Poisson Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: Poisson Encoding,trial, best_score: {study.best_value}")